In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import sklearn

# Информация о датасете

Датасет содержит информацию о фильмах и сериалах на платформе Netflix, а именно: название, жанры, теги, дата выпуска, режиссеров, а также некоторые числовые данные, например выручка, оценки на кино-сервсах. Автор датасета придумал метрику, названную "Hidden Gem Score", которая отражает, насколько тот или иной фильм или сериал получил высокий рейтинг при малом количестве отзывов о нем. Цель работы: решить задачу классификации на заданном датасете. Для этого был создан дополнительный столбец "huge_boxoffice", который показывает, больше 40 миллионов долларов(1) или нет(0) общие сборы фильма или сериала.

## Описание столбцов

 1   Title  - название
 2   Genre - жанры
 3   Tags - теги для поиска                   
 4   Languages - языки              
 5   Series or Movie - сериал или фильм       
 6   Hidden Gem Score - авторская характеристика, описанная в разделе выше      
 7   Country Availability - страны, в которых доступно произведение
 8   Runtime - длительность               
 9   Director - режиссер             
 10  Writer - сценарист                 
 11  Actors - актеры                 
 12  View Rating - возрастной рейтинг            
 13  IMDb Score - средний балл по отзывам на IMDb            
 14  Rotten Tomatoes Score - средний балл по отзывам на Rotten Tomatoes   
 15  Metacritic Score - средний балл по отзывам на Metacritic      
 16  Awards Received - количество полученных наград        
 17  Awards Nominated For - количество номинаций на награды
 18  Boxoffice - общие сборы          
 19  Release Date - дата выхода            
 20  Netflix Release Date - дата выхода на Netflix 
 21  Production House - продюсерская компания      
 22  IMDb Votes - количество отзывов на IMDb         
 23  huge_boxoffice - дополнительная характеристика, по которой производится классификация(описание ниже)

# Загрузка данных

In [2]:
url = "https://media.githubusercontent.com/media/arteboss/ml_course/main/netflix-rotten-tomatoes-metacritic-imdb.csv"
data_raw = pd.read_csv(url)
data_raw.shape

(15480, 29)

Как видно из вывода shape, количество строк датасета равно 15480, а количество столбцов - 29.

# Первичные преобразования датасета

In [3]:
data_raw["Boxoffice"] = data_raw["Boxoffice"].apply(lambda v: float(str(v).replace(",", "").replace("$", "")))

Из-за того, что автор датасета в столбце "Boxoffice" использовал символы '$' и ',', значение в каждой строке этого стобца не распознается как число. Исправим это, применив замену к каждому такому элементу.

In [4]:
data_raw["huge_boxoffice"] = np.where(data_raw["Boxoffice"] >= 4e+07, 1, 0)

Здесь вводим столбец, по которому в будущем будет производиться классификация, а именно, если суммарные сборы фильма/сериала больше 40 миллионов долларов, значение равно 1, иначе 0.

In [5]:
data_raw.drop(['Image', 'Poster', 'TMDb Trailer', 'Trailer Site', 'IMDb Link', 'Netflix Link', 'Summary'], axis=1, inplace=True)

Здесь удаляем столбцы, данные с которых практически бесполезны, так как содержат ссылки, изображения и описание фильма/сериала. Затем выведем первые 10 строк преобразованного датасета методом head():

In [6]:
data_raw.head()

Title                                   Genre  \
0     Lets Fight Ghost  Crime, Drama, Fantasy, Horror, Romance   
1  HOW TO BUILD A GIRL                                  Comedy   
2           Centigrade                         Drama, Thriller   
3                ANNE+                                   Drama   
4                Moxie                 Animation, Short, Drama   

                                                Tags         Languages  \
0  Comedy Programmes,Romantic TV Comedies,Horror ...  Swedish, Spanish   
1       Dramas,Comedies,Films Based on Books,British           English   
2                                          Thrillers           English   
3        TV Dramas,Romantic TV Dramas,Dutch TV Shows           Turkish   
4  Social Issue Dramas,Teen Movies,Dramas,Comedie...           English   

  Series or Movie  Hidden Gem Score  \
0          Series               4.3   
1           Movie               7.0   
2           Movie               6.4   
3          Series               7.7   
4           Movie               8.1   

                                Country Availability       Runtime  \
0                                           Thailand  < 30 minutes   
1                                             Canada      1-2 hour   
2                                             Canada      1-2 hour   
3                                Belgium,Netherlands  < 30 minutes   
4  Lithuania,Poland,France,Iceland,Italy,Spain,Gr...      1-2 hour   

          Director                      Writer  ... Rotten Tomatoes Score  \
0  Tomas Alfredson       John Ajvide Lindqvist  ...                  98.0   
1    Coky Giedroyc               Caitlin Moran  ...                  79.0   
2    Brendan Walsh  Brendan Walsh, Daley Nixon  ...                   NaN   
3              NaN                         NaN  ...                   NaN   
4    Stephen Irwin                         NaN  ...                   NaN   

  Metacritic Score  Awards Received  Awards Nominated For  Boxoffice  \
0             82.0             74.0                  57.0  2122065.0   
1             69.0              1.0                   NaN    70632.0   
2             46.0              NaN                   NaN    16263.0   
3              NaN              1.0                   NaN        NaN   
4              NaN              NaN                   4.0        NaN   

   Release Date  Netflix Release Date                        Production House  \
0   12 Dec 2008            2021-03-04               Canal+, Sandrew Metronome   
1   08 May 2020            2021-03-04  Film 4, Monumental Pictures, Lionsgate   
2   28 Aug 2020            2021-03-04                                     NaN   
3   01 Oct 2016            2021-03-04                                     NaN   
4   22 Sep 2011            2021-03-04                                     NaN   

  IMDb Votes huge_boxoffice  
0   205926.0              0  
1     2838.0              0  
2     1720.0              0  
3     1147.0              0  
4       63.0              0  

[5 rows x 23 columns]

Посмотрим информацию по каждому столбцу, здесь мы можем узнать тип значений в них:

In [7]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15480 entries, 0 to 15479
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Title                  15480 non-null  object 
 1   Genre                  13770 non-null  object 
 2   Tags                   15413 non-null  object 
 3   Languages              13545 non-null  object 
 4   Series or Movie        15480 non-null  object 
 5   Hidden Gem Score       13379 non-null  float64
 6   Country Availability   15461 non-null  object 
 7   Runtime                15479 non-null  object 
 8   Director               10772 non-null  object 
 9   Writer                 11150 non-null  object 
 10  Actors                 13555 non-null  object 
 11  View Rating            8456 non-null   object 
 12  IMDb Score             13381 non-null  float64
 13  Rotten Tomatoes Score  6382 non-null   float64
 14  Metacritic Score       4336 non-null   float64
 15  Aw

## Обработка категориальных признаков

Признаки "Series or Movie", "Runtime" категориальные и имеют всего 2 и 4 вида значений соответственно. Обработаем их, преобразовав их тип к специальному типу, нужному для обработки категориальных значений:

In [8]:
data_raw['Series or Movie'] = data_raw['Series or Movie'].astype('category')
data_raw['Runtime'] = data_raw['Runtime'].astype('category')

In [9]:
data_raw['Series or Movie'].dtype

CategoricalDtype(categories=['Movie', 'Series'], ordered=False)

In [10]:
data_raw['Runtime'].dtype

CategoricalDtype(categories=['1-2 hour', '30-60 mins', '< 30 minutes', '> 2 hrs'], ordered=False)

# Визуализация и получение основных характеристик

Подсчитаем основные характеристики числовых признаков, используя метод describe():

In [11]:
data_raw.describe()

Hidden Gem Score    IMDb Score  Rotten Tomatoes Score  \
count      13379.000000  13381.000000            6382.000000   
mean           5.937551      6.496054              59.523034   
std            2.250202      1.146910              26.999173   
min            0.600000      1.000000               0.000000   
25%            3.800000      5.800000              38.000000   
50%            6.800000      6.600000              64.000000   
75%            7.900000      7.300000              83.000000   
max            9.800000      9.700000             100.000000   

       Metacritic Score  Awards Received  Awards Nominated For     Boxoffice  \
count       4336.000000      6075.000000           7661.000000  4.007000e+03   
mean          56.813653         8.764444             13.983161  4.568229e+07   
std           17.582545        18.311171             29.821052  7.097634e+07   
min            5.000000         1.000000              1.000000  7.200000e+01   
25%           44.000000         1.000000              2.000000  7.816345e+05   
50%           57.000000         3.000000              5.000000  1.955107e+07   
75%           70.000000         8.000000             12.000000  5.969536e+07   
max          100.000000       300.000000            386.000000  6.593639e+08   

         IMDb Votes  huge_boxoffice  
count  1.337900e+04    15480.000000  
mean   4.272841e+04        0.090698  
std    1.257012e+05        0.287188  
min    5.000000e+00        0.000000  
25%    4.035000e+02        0.000000  
50%    2.322000e+03        0.000000  
75%    2.089050e+04        0.000000  
max    2.354197e+06        1.000000

Построим матрицу корреляции, чтобы увидеть, какие признаки хорошо коррелируют:

In [12]:
corr_mat = data_raw.corr()
corr_mat

Hidden Gem Score  IMDb Score  Rotten Tomatoes Score  \
Hidden Gem Score               1.000000    0.120373               0.397905   
IMDb Score                     0.120373    1.000000               0.710922   
Rotten Tomatoes Score          0.397905    0.710922               1.000000   
Metacritic Score               0.521470    0.717715               0.903345   
Awards Received               -0.112787    0.291142               0.301380   
Awards Nominated For          -0.159500    0.289843               0.309572   
Boxoffice                     -0.211815    0.138257               0.053706   
IMDb Votes                    -0.359540    0.229760               0.181762   
huge_boxoffice                -0.437633    0.054761              -0.002739   

                       Metacritic Score  Awards Received  \
Hidden Gem Score               0.521470        -0.112787   
IMDb Score                     0.717715         0.291142   
Rotten Tomatoes Score          0.903345         0.301380   
Metacritic Score               1.000000         0.431420   
Awards Received                0.431420         1.000000   
Awards Nominated For           0.449726         0.817262   
Boxoffice                      0.061548         0.193976   
IMDb Votes                     0.261384         0.469818   
huge_boxoffice                 0.009255         0.176646   

                       Awards Nominated For  Boxoffice  IMDb Votes  \
Hidden Gem Score                  -0.159500  -0.211815   -0.359540   
IMDb Score                         0.289843   0.138257    0.229760   
Rotten Tomatoes Score              0.309572   0.053706    0.181762   
Metacritic Score                   0.449726   0.061548    0.261384   
Awards Received                    0.817262   0.193976    0.469818   
Awards Nominated For               1.000000   0.284885    0.497690   
Boxoffice                          0.284885   1.000000    0.630301   
IMDb Votes                         0.497690   0.630301    1.000000   
huge_boxoffice                     0.226649   0.690545    0.560818   

                       huge_boxoffice  
Hidden Gem Score            -0.437633  
IMDb Score                   0.054761  
Rotten Tomatoes Score       -0.002739  
Metacritic Score             0.009255  
Awards Received              0.176646  
Awards Nominated For         0.226649  
Boxoffice                    0.690545  
IMDb Votes                   0.560818  
huge_boxoffice               1.000000

Видим, что созданный вручную признак "huge_boxoffice" хорошо коррелирует с признаками "Boxoffice"(неудивительно, ведь он создан только из него) и "IMDb Votes". 

Построим графики зависимости числовых признаков друг от друга попарно:

In [13]:
np.random.seed(42)
random_subset = np.random.choice(np.arange(data_raw.shape[0]), size=1000, replace=False)
sns.pairplot(data_raw.iloc[random_subset], hue = 'huge_boxoffice', diag_kind='hist')
plt.legend()
pass

No handles with labels found to put in legend.


# Основные преобразования с датасетом

## Заполнение пропущенных значений

Для начала посмотрим, какие столбцы содержат пропущенные значения и количество таковых в каждом из них:

In [14]:
data_raw.isna().sum()

Title                        0
Genre                     1710
Tags                        67
Languages                 1935
Series or Movie              0
Hidden Gem Score          2101
Country Availability        19
Runtime                      1
Director                  4708
Writer                    4330
Actors                    1925
View Rating               7024
IMDb Score                2099
Rotten Tomatoes Score     9098
Metacritic Score         11144
Awards Received           9405
Awards Nominated For      7819
Boxoffice                11473
Release Date              2107
Netflix Release Date         0
Production House         10331
IMDb Votes                2101
huge_boxoffice               0
dtype: int64

Чтобы не изменить среднее значение по столбцу для числовых признаков, самый распространенный способ - заполнить пропущенные значения медианой:

In [15]:
data_raw["Hidden Gem Score"].fillna(data_raw["Hidden Gem Score"].median(axis = 0), axis=0 , inplace=True)
data_raw["IMDb Score"].fillna(data_raw["IMDb Score"].median(axis = 0), axis=0 , inplace=True)
data_raw["Rotten Tomatoes Score"].fillna(data_raw["Rotten Tomatoes Score"].median(axis = 0), axis=0 , inplace=True)
data_raw["Metacritic Score"].fillna(data_raw["Metacritic Score"].median(axis = 0), axis=0 , inplace=True)
data_raw["Awards Received"].fillna(data_raw["Awards Received"].median(axis = 0), axis=0 , inplace=True)
data_raw["Awards Nominated For"].fillna(data_raw["Awards Nominated For"].median(axis = 0), axis=0 , inplace=True)
data_raw["Boxoffice"].fillna(data_raw["Boxoffice"].median(axis = 0), axis=0 , inplace=True)
data_raw["IMDb Votes"].fillna(data_raw["IMDb Votes"].median(axis = 0), axis=0 , inplace=True)

In [16]:
data_raw.isna().sum()

Title                        0
Genre                     1710
Tags                        67
Languages                 1935
Series or Movie              0
Hidden Gem Score             0
Country Availability        19
Runtime                      1
Director                  4708
Writer                    4330
Actors                    1925
View Rating               7024
IMDb Score                   0
Rotten Tomatoes Score        0
Metacritic Score             0
Awards Received              0
Awards Nominated For         0
Boxoffice                    0
Release Date              2107
Netflix Release Date         0
Production House         10331
IMDb Votes                   0
huge_boxoffice               0
dtype: int64

## Обработка категориальных признаков(бинаризация)

Введем дополнительные столбцы, которые имеют всего два значения 0 или 1, и заменим столбцы с категориальными признаками на созданные:

In [17]:
dummies = data_raw['Genre'].str.join(sep='').str.get_dummies(sep=', ')
dummies_1 = pd.get_dummies(data_raw['Series or Movie'])
dummies_2 = pd.get_dummies(data_raw['Runtime'])

In [18]:
data_raw = pd.concat((data_raw, dummies), axis=1)
data_raw = pd.concat((data_raw, dummies_1), axis=1)
data_raw = pd.concat((data_raw, dummies_2), axis=1)
data_raw = data_raw.drop(['Genre', 'Series or Movie', 'Runtime'], axis=1)
data_raw.head()

Title                                               Tags  \
0     Lets Fight Ghost  Comedy Programmes,Romantic TV Comedies,Horror ...   
1  HOW TO BUILD A GIRL       Dramas,Comedies,Films Based on Books,British   
2           Centigrade                                          Thrillers   
3                ANNE+        TV Dramas,Romantic TV Dramas,Dutch TV Shows   
4                Moxie  Social Issue Dramas,Teen Movies,Dramas,Comedie...   

          Languages  Hidden Gem Score  \
0  Swedish, Spanish               4.3   
1           English               7.0   
2           English               6.4   
3           Turkish               7.7   
4           English               8.1   

                                Country Availability         Director  \
0                                           Thailand  Tomas Alfredson   
1                                             Canada    Coky Giedroyc   
2                                             Canada    Brendan Walsh   
3                                Belgium,Netherlands              NaN   
4  Lithuania,Poland,France,Iceland,Italy,Spain,Gr...    Stephen Irwin   

                       Writer  \
0       John Ajvide Lindqvist   
1               Caitlin Moran   
2  Brendan Walsh, Daley Nixon   
3                         NaN   
4                         NaN   

                                              Actors View Rating  IMDb Score  \
0  Kåre Hedebrant, Per Ragnar, Lina Leandersson, ...           R         7.9   
1  Paddy Considine, Cleo, Beanie Feldstein, Dónal...           R         5.8   
2                  Genesis Rodriguez, Vincent Piazza     Unrated         4.3   
3  Vahide Perçin, Gonca Vuslateri, Cansu Dere, Be...         NaN         6.5   
4                                       Ragga Gudrun         NaN         6.3   

   ...  Talk-Show  Thriller  War  Western  Movie Series 1-2 hour 30-60 mins  \
0  ...          0         0    0        0      0      1        0          0   
1  ...          0         0    0        0      1      0        1          0   
2  ...          0         1    0        0      1      0        1          0   
3  ...          0         0    0        0      0      1        0          0   
4  ...          0         0    0        0      1      0        1          0   

   < 30 minutes  > 2 hrs  
0             1        0  
1             0        0  
2             0        0  
3             1        0  
4             0        0  

[5 rows x 54 columns]

Удалим также остальные столбцы, имеющие тип "object":

In [19]:
data_raw.drop(['Title', 'Tags', 'Languages',
       'Country Availability', 'Director', 'Writer', 'Actors', 'View Rating', 'Release Date',
       'Netflix Release Date', 'Production House'], axis=1, inplace = True)

In [20]:
data_raw.head()

Hidden Gem Score  IMDb Score  Rotten Tomatoes Score  Metacritic Score  \
0               4.3         7.9                   98.0              82.0   
1               7.0         5.8                   79.0              69.0   
2               6.4         4.3                   64.0              46.0   
3               7.7         6.5                   64.0              57.0   
4               8.1         6.3                   64.0              57.0   

   Awards Received  Awards Nominated For   Boxoffice  IMDb Votes  \
0             74.0                  57.0   2122065.0    205926.0   
1              1.0                   5.0     70632.0      2838.0   
2              3.0                   5.0     16263.0      1720.0   
3              1.0                   5.0  19551067.0      1147.0   
4              3.0                   4.0  19551067.0        63.0   

   huge_boxoffice  Action  ...  Talk-Show  Thriller  War  Western  Movie  \
0               0       0  ...          0         0    0        0      0   
1               0       0  ...          0         0    0        0      1   
2               0       0  ...          0         1    0        0      1   
3               0       0  ...          0         0    0        0      0   
4               0       0  ...          0         0    0        0      1   

   Series  1-2 hour  30-60 mins  < 30 minutes  > 2 hrs  
0       1         0           0             1        0  
1       0         1           0             0        0  
2       0         1           0             0        0  
3       1         0           0             1        0  
4       0         1           0             0        0  

[5 rows x 43 columns]

## Нормализация

In [21]:
data_raw = (data_raw - data_raw.min(axis=0)) / (data_raw.max(axis=0) - data_raw.min(axis=0))
data_raw.describe()

Hidden Gem Score    IMDb Score  Rotten Tomatoes Score  \
count      15480.000000  15480.000000           15480.000000   
mean           0.592892      0.633350               0.621543   
std            0.229639      0.122633               0.174745   
min            0.000000      0.000000               0.000000   
25%            0.369565      0.574713               0.640000   
50%            0.673913      0.643678               0.640000   
75%            0.782609      0.712644               0.640000   
max            1.000000      1.000000               1.000000   

       Metacritic Score  Awards Received  Awards Nominated For     Boxoffice  \
count      15480.000000     15480.000000          15480.000000  15480.000000   
mean           0.546819         0.014255              0.021937      0.039910   
std            0.097949         0.039501              0.055723      0.057447   
min            0.000000         0.000000              0.000000      0.000000   
25%            0.547368         0.006689              0.010390      0.029651   
50%            0.547368         0.006689              0.010390      0.029651   
75%            0.547368         0.006689              0.010390      0.029651   
max            1.000000         1.000000              1.000000      1.000000   

         IMDb Votes  huge_boxoffice        Action  ...    Talk-Show  \
count  15480.000000    15480.000000  15480.000000  ...  15480.00000   
mean       0.015818        0.090698      0.181525  ...      0.00168   
std        0.049986        0.287188      0.385465  ...      0.04095   
min        0.000000        0.000000      0.000000  ...      0.00000   
25%        0.000232        0.000000      0.000000  ...      0.00000   
50%        0.000984        0.000000      0.000000  ...      0.00000   
75%        0.005924        0.000000      0.000000  ...      0.00000   
max        1.000000        1.000000      1.000000  ...      1.00000   

           Thriller           War       Western         Movie        Series  \
count  15480.000000  15480.000000  15480.000000  15480.000000  15480.000000   
mean       0.176938      0.021318      0.007041      0.755620      0.244380   
std        0.381629      0.144446      0.083619      0.429733      0.429733   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      1.000000      0.000000   
50%        0.000000      0.000000      0.000000      1.000000      0.000000   
75%        0.000000      0.000000      0.000000      1.000000      0.000000   
max        1.000000      1.000000      1.000000      1.000000      1.000000   

           1-2 hour    30-60 mins  < 30 minutes       > 2 hrs  
count  15480.000000  15480.000000  15480.000000  15480.000000  
mean       0.589212      0.021576      0.258140      0.131008  
std        0.491993      0.145300      0.437625      0.337420  
min        0.000000      0.000000      0.000000      0.000000  
25%        0.000000      0.000000      0.000000      0.000000  
50%        1.000000      0.000000      0.000000      0.000000  
75%        1.000000      0.000000      1.000000      0.000000  
max        1.000000      1.000000      1.000000      1.000000  

[8 rows x 43 columns]

# Классификация

Для оставшихся признаков проведем исследования, используя алгоритмы KNN и RandomForest:

In [22]:
X = data_raw.drop(['Boxoffice', 'huge_boxoffice'], axis=1) 
y = data_raw['huge_boxoffice']

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 

N_train, N_test

(11610, 3870)

## KNN

### 5 соседей(по умолчанию)

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn_model = KNeighborsClassifier()
knn_model.fit(X_train.values, y_train.values)
y_train_pred = knn_model.predict(X_train.values)
y_test_pred = knn_model.predict(X_test.values)
"Train: {}, Test: {}".format(accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred))

'Train: 0.9443583118001723, Test: 0.9175710594315245'

### 10 соседей

In [25]:
knn_model = KNeighborsClassifier(n_neighbors=10)
knn_model.fit(X_train.values, y_train.values)
y_train_pred = knn_model.predict(X_train.values)
y_test_pred = knn_model.predict(X_test.values)
"Train: {}, Test: {}".format(accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred))

'Train: 0.9306632213608957, Test: 0.9227390180878553'

### 7 соседей

In [26]:
knn_model = KNeighborsClassifier(n_neighbors=7)
knn_model.fit(X_train.values, y_train.values)
y_train_pred = knn_model.predict(X_train.values)
y_test_pred = knn_model.predict(X_test.values)
"Train: {}, Test: {}".format(accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred))

'Train: 0.9375538329026701, Test: 0.9183462532299742'

Исследования точности показали, что самым точным алгоритм получился для 10 соседей.

## Random Forest

In [27]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model.fit(X_train.values, y_train.values)
y_train_pred = rf_model.predict(X_train.values)
y_test_pred = rf_model.predict(X_test.values)
"Train: {}, Test: {}".format(accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred))

'Train: 1.0, Test: 0.9607235142118863'

In [28]:
rf_model = RandomForestClassifier(n_estimators=500)
rf_model.fit(X_train.values, y_train.values)
y_train_pred = rf_model.predict(X_train.values)
y_test_pred = rf_model.predict(X_test.values)
"Train: {}, Test: {}".format(accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred))

'Train: 1.0, Test: 0.9594315245478037'

# Выводы
* На тестовой и тренировочной выборках по точности выигрывает алгоритм Random Forest;
* При увеличении числа соседей у KNN точность увеличивается;
* При увеличении числа решающих деревьев у Random Forest точность увеличивается.